# Projet

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import files
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import io
from tensorflow import keras
from tensorflow.keras import layers
import keras.preprocessing.text
import tensorflow.keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import multilabel_confusion_matrix as mcm
import tensorflow_hub as hub
tf.keras.backend.clear_session()
np.set_printoptions(precision=3, suppress=True)
pd.options.display.max_colwidth = 1000

Using TensorFlow backend.


In [3]:
print(tf.__version__)

2.0.0


In [4]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Read CSV

In [0]:
Total_data_for_tokenizer=pd.read_csv('/content/gdrive/My Drive/Data/Total_data_for_tokenizer.csv' , sep='\t')

In [0]:
training_set_TripAdvisor=pd.read_csv('/content/gdrive/My Drive/Data/training_set_TripAdvisor.csv' , sep='\t')

In [0]:
test_set_TripAdvisor=pd.read_csv('/content/gdrive/My Drive/Data/test_set_TripAdvisor.csv' , sep='\t')

In [0]:
training_set_IMDB=pd.read_csv('/content/gdrive/My Drive/Data/training_set_IMDB.csv' , sep='\t')

In [0]:
test_set_IMDB=pd.read_csv('/content/gdrive/My Drive/Data/test_set_IMDB.csv' , sep='\t')

In [0]:
#data_tfds_IMDB=pd.read_csv('/content/gdrive/My Drive/Data/data_tfds_IMDB.csv', sep='\t')

In [0]:
#all_data_TripAdvisor=pd.read_csv('/content/gdrive/My Drive/Data/all_data_TripAdvisor.csv', sep='\t')

In [0]:
del Total_data_for_tokenizer['Unnamed: 0']
del training_set_TripAdvisor['Unnamed: 0']
del test_set_TripAdvisor['Unnamed: 0']
del training_set_IMDB['Unnamed: 0']
del test_set_IMDB['Unnamed: 0']

# Tokenizer

In [0]:
def create_dataset(dataframe):
  msk = np.random.rand(len(dataframe)) < 0.8

  train = dataframe[msk]

  test = dataframe[~msk]
  #print(train_file_path.head)
  all_data = np.array(dataframe.values.tolist())
  training_set = np.array(train.values.tolist())
  test_set=np.array(test.values.tolist())
  print("Dataset Length: ",len(training_set)+ len(test_set))
  return all_data, training_set, test_set

In [0]:
def create_tokenizer(dataset):
  # create the tokenizer
  #max_features = 6000
  #tokenizer = Tokenizer(num_words=max_features , filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token='UNK')
  
  tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token='UNK')
  # fit the tokenizer on the documents
  tokenizer.fit_on_texts(dataset[:,1])
  # summarize what was learned
  print("Word counter: ",tokenizer.word_counts)
  print("Number of sentences: ",tokenizer.document_count)
  print("Word Index: ", tokenizer.word_index)
  print("Word Docs: ",tokenizer.word_docs)
  return tokenizer

In [0]:
def vocab_size(tokenizer):
  voca_size=len(tokenizer.word_counts)+2   # 1 ?
  return voca_size

In [0]:
def maxlen(data):
  return max([len(x) for x in data])

In [0]:
Total_data_for_tokenizer_np = np.array(Total_data_for_tokenizer.values.tolist())

In [18]:
tokenizer=create_tokenizer(Total_data_for_tokenizer_np)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
def tokenize_sentences(input_set):
  input_sequences = []
  for line in input_set[:,1]:
      token_list = tokenizer.texts_to_sequences([line])[0]
      input_sequences.append(token_list)
  return input_sequences

In [0]:
training_set_IMDB_np = np.array(training_set_IMDB.values.tolist())
test_set_IMDB_np = np.array(test_set_IMDB.values.tolist())
training_set_TripAdvisor_np = np.array(training_set_TripAdvisor.values.tolist())
test_set_TripAdvisor_np = np.array(test_set_TripAdvisor.values.tolist())

In [0]:
input_training_set_IMDB = tokenize_sentences(training_set_IMDB_np)
input_test_set_IMDB = tokenize_sentences(test_set_IMDB_np)
input_training_set_TripAdvisor = tokenize_sentences(training_set_TripAdvisor_np)
input_test_set_TripAdvisor = tokenize_sentences(test_set_TripAdvisor_np)

In [0]:
def pad_sentences(input_sequences, max_sequence_len):
  input_padded_sequences = np.array(pad_sequences(input_sequences,   
                            maxlen=max_sequence_len, padding='pre'))
  return input_padded_sequences

In [0]:
def pad_sentences_post(input_sequences, max_sequence_len):
  input_padded_sequences = np.array(pad_sequences(input_sequences,   
                            maxlen=max_sequence_len, padding='post'))
  return input_padded_sequences

In [0]:
def maxlen(data1, data2 , data3 , data4):
  max_len_data1 = max([len(x) for x in data1])
  max_len_data2 = max([len(x) for x in data2])
  max_len_data3 = max([len(x) for x in data3])
  max_len_data4 = max([len(x) for x in data4])
  return max(max_len_data1, max_len_data2 , max_len_data3 , max_len_data4)

In [0]:
max_len = maxlen(input_training_set_IMDB, input_test_set_IMDB, input_training_set_TripAdvisor, input_test_set_TripAdvisor)

#Embeddings pretraining (Next Word Prediction)


In [26]:
'''def split_sentences_forward(input_set):
  input_sequences = []
  for line in input_set[:,1]:
      token_list = tokenizer.texts_to_sequences([line])[0]
      for i in range(1, len(line)):
        if i < 99 :
          n_gram_sequence = token_list[:i+1]
        else :
          n_gram_sequence = token_list[i-99:i+1]
        input_sequences.append(n_gram_sequence)
  return input_sequences'''

'def split_sentences_forward(input_set):\n  input_sequences = []\n  for line in input_set[:,1]:\n      token_list = tokenizer.texts_to_sequences([line])[0]\n      for i in range(1, len(line)):\n        if i < 99 :\n          n_gram_sequence = token_list[:i+1]\n        else :\n          n_gram_sequence = token_list[i-99:i+1]\n        input_sequences.append(n_gram_sequence)\n  return input_sequences'

In [0]:
def split_sentences_forward(input_set):
  input_sequences = []
  for line in input_set:
      for i in range(1, len(line)):
        if i < 99 :
          n_gram_sequence = line[:i+1]
        else :
          n_gram_sequence = line[i-99:i+1]
        input_sequences.append(n_gram_sequence)
  return input_sequences

In [0]:
def split_sentences_Backward(input_set):
  input_sequences = []
  for line in input_set[:,1]:
      token_list = tokenizer.texts_to_sequences([line])[0]
      for i in range(1, len(token_list)):
        if i < 99 :
          n_gram_sequence = token_list[:i+1]
        else :
          n_gram_sequence = token_list[i-99:i+1]
        input_sequences.append(n_gram_sequence)
  return input_sequences

In [0]:
def create_dataset_slices_embaddings(input_sequences):
  predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
  #label = tf.keras.utils.to_categorical(label, num_classes=voc_size)
  return predictors, label

In [0]:
def create_preprocessed_dataset(predictors, labels):
  dataset = tf.data.Dataset.from_tensor_slices((predictors, labels))
  return dataset

In [0]:
def top_k_categorical_accuracy1(y_true, y_pred, k=10):
    return K.mean(K.in_top_k(K.cast(y_pred,dtype='float32'),K.argmax(y_true, axis=-1), k), axis=-1)

In [0]:
input_train_WE_1=split_sentences_forward(input_training_set_IMDB)

In [0]:
input_train_WE_2=split_sentences_forward(input_training_set_TripAdvisor)

In [0]:
input_test_WE_1=split_sentences_forward(input_test_set_IMDB[5000:])

In [0]:
input_test_WE_2=split_sentences_forward(input_test_set_TripAdvisor)

In [0]:
input_train_WE_sequences_1 = pad_sentences(input_train_WE_1, 100)
input_train_WE_sequences_2 = pad_sentences(input_train_WE_2, 100)
input_test_WE_sequences_1 = pad_sentences(input_test_WE_1, 100)
input_test_WE_sequences_2 = pad_sentences(input_test_WE_2, 100)

In [0]:
training_predictors_WE, training_label_WE=create_dataset_slices_embaddings(input_train_WE_sequences_1)
test_predictors_WE, test_label_WE= create_dataset_slices_embaddings(input_test_WE_sequences_1)

In [0]:
training_predictors_WE_2, training_label_WE_2=create_dataset_slices_embaddings(input_train_WE_sequences_2)
test_predictors_WE_2, test_label_WE_2= create_dataset_slices_embaddings(input_test_WE_sequences_2)

In [0]:
voc_size = vocab_size(tokenizer)

In [40]:
voc_size

80008

In [0]:
max_features = 6000

In [0]:
#EmbeddingLayer = tf.keras.layers.Embedding(max_features, 64)
EmbeddingLayer = tf.keras.layers.Embedding(voc_size, 64)
LstmLayer = tf.keras.layers.LSTM(150)
DenseLayerEmbedding = tf.keras.layers.Dense(64, activation='relu')
DropLayerEmbedding = tf.keras.layers.Dropout(0.1)
DenseLayerOutputEmbedding = tf.keras.layers.Dense(voc_size, activation='softmax')

In [43]:
model = tf.keras.Sequential()
model.add(EmbeddingLayer)
model.add(LstmLayer)
model.add(DenseLayerEmbedding)
model.add(DropLayerEmbedding)
model.add(DenseLayerOutputEmbedding)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          5120512   
_________________________________________________________________
lstm (LSTM)                  (None, 150)               129000    
_________________________________________________________________
dense (Dense)                (None, 64)                9664      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 80008)             5200520   
Total params: 10,459,696
Trainable params: 10,459,696
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [0]:
checkpoint_path = "training_Embadding/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

In [46]:
history = model.fit(training_predictors_WE, 
                    training_label_WE,
                    epochs=3,
                    batch_size=100,
                    validation_data=(test_predictors_WE, test_label_WE),
                    verbose=1,
                    callbacks=[cp_callback])

Train on 3616784 samples, validate on 2831359 samples
Epoch 1/3
3616700/3616784 [============================>.] - ETA: 0s - loss: 7.6890 - accuracy: 0.0601
Epoch 00001: saving model to training_Embadding/cp-0001.ckpt
3616784/3616784 [==============================] - 3023s 836us/sample - loss: 7.6890 - accuracy: 0.0601 - val_loss: 7.5776 - val_accuracy: 0.0713
Epoch 2/3
1828800/3616784 [==============>...............] - ETA: 20:11 - loss: 7.4246 - accuracy: 0.0731Buffered data was truncated after reaching the output size limit.

In [47]:
history = model.fit(training_predictors_WE_2, 
                    training_label_WE_2,
                    epochs=5,
                    batch_size=100,
                    validation_data=(test_predictors_WE_2, test_label_WE_2),
                    verbose=1,
                    callbacks=[cp_callback])

Train on 947983 samples, validate on 237895 samples
Epoch 1/5
947900/947983 [============================>.] - ETA: 0s - loss: 5.7881 - accuracy: 0.1432
Epoch 00001: saving model to training_Embadding/cp-0001.ckpt
947983/947983 [==============================] - 710s 749us/sample - loss: 5.7880 - accuracy: 0.1432 - val_loss: 5.1368 - val_accuracy: 0.1876
Epoch 2/5
947900/947983 [============================>.] - ETA: 0s - loss: 5.0393 - accuracy: 0.1938
Epoch 00002: saving model to training_Embadding/cp-0002.ckpt
947983/947983 [==============================] - 712s 751us/sample - loss: 5.0393 - accuracy: 0.1938 - val_loss: 4.8432 - val_accuracy: 0.2122
Epoch 3/5
947900/947983 [============================>.] - ETA: 0s - loss: 4.7762 - accuracy: 0.2128
Epoch 00003: saving model to training_Embadding/cp-0003.ckpt
947983/947983 [==============================] - 714s 753us/sample - loss: 4.7762 - accuracy: 0.2128 - val_loss: 4.7101 - val_accuracy: 0.2229
Epoch 4/5
947900/947983 [========

#BI-LSTM

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
input_train_CL_sequences_1 = pad_sentences_post(input_training_set_IMDB, max_len)
input_train_CL_sequences_2 = pad_sentences_post(input_training_set_TripAdvisor, max_len)
input_test_CL_sequences_1 = pad_sentences_post(input_test_set_IMDB, max_len)
input_test_CL_sequences_2 = pad_sentences_post(input_test_set_TripAdvisor, max_len)

In [0]:
def create_dataset_slices_sentence_classification(input_sequences, targets):
  predictors= tf.constant(input_sequences)
  targets=targets.astype(int)
  nb_classes = 5
  targets = targets.reshape(-1)
  one_hot_targets = np.eye(nb_classes)[targets-1]
  return predictors, one_hot_targets

In [0]:
training_predictors, training_label=create_dataset_slices_sentence_classification(input_train_CL_sequences_2,training_set_TripAdvisor_np[:,0])
test_predictors, test_label=create_dataset_slices_sentence_classification(input_test_CL_sequences_2,test_set_TripAdvisor_np[:,0])

In [0]:
def create_dataset_slices_sentence_classification_bina(input_sequences, targets):
  predictors= tf.constant(input_sequences)
  targets=targets.astype(int)
  nb_classes = 2
  targets = targets.reshape(-1)
  one_hot_targets = np.eye(nb_classes)[targets-1]
  return predictors, one_hot_targets

In [0]:
training_predictors_tfds, training_label_tfds=create_dataset_slices_sentence_classification_bina(input_train_CL_sequences_1,training_set_IMDB_np[:,0])
test_predictors_tfds, test_label_tfds=create_dataset_slices_sentence_classification_bina(input_test_CL_sequences_1,test_set_IMDB_np[:,0])

In [0]:
BidirectionalLayer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences = True))
GlobalMaxPool1DLayer = tf.keras.layers.GlobalMaxPool1D()
DenseLayer = tf.keras.layers.Dense(20, activation='relu')
DropLayer = tf.keras.layers.Dropout(0.05)
DenseLayerOutput = tf.keras.layers.Dense(2, activation=tf.keras.activations.softmax)

In [0]:
EmbeddingLayer.trainable = False

In [56]:
modelBin = tf.keras.Sequential()
modelBin.add(EmbeddingLayer)
modelBin.add(BidirectionalLayer)
modelBin.add(GlobalMaxPool1DLayer)
modelBin.add(DenseLayer)
modelBin.add(DropLayer)
modelBin.add(DenseLayerOutput)

modelBin.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          5120512   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 64)          24832     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 42        
Total params: 5,146,686
Trainable params: 26,174
Non-trainable params: 5,120,512
_______________________________________

In [0]:
modelBin.compile(loss='categorical_crossentropy', #'mean_squared_error'
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse','mae','accuracy'])

Trois PISTE  mots frequ entre classe ,  max_features = 6000 ===== > tokenizer = Tokenizer(num_words=max_features)

In [58]:
history = modelBin.fit(training_predictors_tfds, 
                    training_label_tfds,
                    epochs=4,
                    batch_size=100,
                    validation_data=(test_predictors_tfds, test_label_tfds),
                    verbose=1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 81s 3ms/sample - loss: 0.5569 - mse: 0.1888 - mae: 0.3877 - accuracy: 0.7118 - val_loss: 0.4789 - val_mse: 0.1573 - val_mae: 0.3103 - val_accuracy: 0.7668
Epoch 2/4
25000/25000 [==============================] - 77s 3ms/sample - loss: 0.4415 - mse: 0.1421 - mae: 0.2925 - accuracy: 0.7946 - val_loss: 0.4238 - val_mse: 0.1365 - val_mae: 0.2870 - val_accuracy: 0.8010
Epoch 3/4
25000/25000 [==============================] - 77s 3ms/sample - loss: 0.3885 - mse: 0.1226 - mae: 0.2531 - accuracy: 0.8288 - val_loss: 0.3781 - val_mse: 0.1193 - val_mae: 0.2578 - val_accuracy: 0.8296
Epoch 4/4
25000/25000 [==============================] - 78s 3ms/sample - loss: 0.3600 - mse: 0.1122 - mae: 0.2329 - accuracy: 0.8448 - val_loss: 0.3655 - val_mse: 0.1155 - val_mae: 0.2370 - val_accuracy: 0.8366


In [0]:
EmbeddingLayer.trainable = True

In [0]:
modelBin.compile(loss='categorical_crossentropy', #'mean_squared_error'
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse','mae','accuracy'])

In [61]:
history = modelBin.fit(training_predictors_tfds, 
                    training_label_tfds,
                    epochs=3,
                    batch_size=100,
                    validation_data=(test_predictors_tfds, test_label_tfds),
                    verbose=1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 95s 4ms/sample - loss: 0.2990 - mse: 0.0913 - mae: 0.1873 - accuracy: 0.8734 - val_loss: 0.2945 - val_mse: 0.0913 - val_mae: 0.1751 - val_accuracy: 0.8723
Epoch 2/3
25000/25000 [==============================] - 92s 4ms/sample - loss: 0.1855 - mse: 0.0533 - mae: 0.1126 - accuracy: 0.9281 - val_loss: 0.2788 - val_mse: 0.0853 - val_mae: 0.1543 - val_accuracy: 0.8838
Epoch 3/3
25000/25000 [==============================] - 91s 4ms/sample - loss: 0.0998 - mse: 0.0266 - mae: 0.0602 - accuracy: 0.9666 - val_loss: 0.2958 - val_mse: 0.0877 - val_mae: 0.1445 - val_accuracy: 0.8811


In [0]:
DenseLayerOutput_5 = tf.keras.layers.Dense(5, activation=tf.keras.activations.softmax)

In [63]:
model1 = tf.keras.Sequential()
model1.add(EmbeddingLayer)
model1.add(BidirectionalLayer)
model1.add(GlobalMaxPool1DLayer)
model1.add(DenseLayer)
model1.add(DropLayer)
model1.add(DenseLayerOutput_5)

model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          5120512   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 64)          24832     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 105       
Total params: 5,146,749
Trainable params: 5,146,749
Non-trainable params: 0
____________________________________________

In [0]:
model1.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse','mae','accuracy'])

In [65]:
history = model1.fit(training_predictors, 
                    training_label,
                    epochs=5,
                    batch_size=100,
                    validation_data=(test_predictors, test_label),
                    verbose=1)

Train on 7984 samples, validate on 2016 samples
Epoch 1/5
7984/7984 [==============================] - 27s 3ms/sample - loss: 1.3373 - mse: 0.1370 - mae: 0.2798 - accuracy: 0.4305 - val_loss: 1.1658 - val_mse: 0.1211 - val_mae: 0.2487 - val_accuracy: 0.5015
Epoch 2/5
7984/7984 [==============================] - 23s 3ms/sample - loss: 1.0715 - mse: 0.1138 - mae: 0.2342 - accuracy: 0.5410 - val_loss: 1.0087 - val_mse: 0.1078 - val_mae: 0.2254 - val_accuracy: 0.5779
Epoch 3/5
7984/7984 [==============================] - 23s 3ms/sample - loss: 0.9366 - mse: 0.1028 - mae: 0.2112 - accuracy: 0.5976 - val_loss: 0.9452 - val_mse: 0.1017 - val_mae: 0.2047 - val_accuracy: 0.6066
Epoch 4/5
7984/7984 [==============================] - 23s 3ms/sample - loss: 0.8429 - mse: 0.0942 - mae: 0.1939 - accuracy: 0.6465 - val_loss: 0.9088 - val_mse: 0.0997 - val_mae: 0.2016 - val_accuracy: 0.6091
Epoch 5/5
7984/7984 [==============================] - 23s 3ms/sample - loss: 0.7666 - mse: 0.0864 - mae: 0.1789